# Обзор
В этом соревновании перед вами стоит задача разработать модель, способную предсказать движение цен на закрытие для сотен акций, котирующихся на бирже Nasdaq, используя данные из книги заказов и аукциона на закрытие акции. Информация с аукциона может быть использована для корректировки цен, оценки динамики спроса и предложения и определения торговых возможностей.

## Описание

- Фондовые биржи - это быстро развивающаяся среда с высокими ставками, где важна каждая секунда. Напряженность возрастает по мере приближения торгового дня к концу, достигая пика в критические последние десять минут. Эти моменты, часто характеризующиеся повышенной волатильностью и быстрыми колебаниями цен, играют ключевую роль в формировании глобальной экономической картины дня.

- Каждый торговый день на фондовой бирже Nasdaq завершается закрытием перекрестного аукциона Nasdaq. Этот процесс устанавливает официальные цены закрытия для ценных бумаг, котирующихся на бирже. Эти цены закрытия служат ключевыми показателями для инвесторов, аналитиков и других участников рынка при оценке эффективности отдельных ценных бумаг и рынка в целом.

- В рамках этого сложного финансового ландшафта работает Optiver, ведущий мировой производитель электронных товаров. Благодаря технологическим инновациям Optiver торгует широким спектром финансовых инструментов, таких как деривативы, наличные акции, ETF, облигации и иностранная валюта, предлагая конкурентоспособные двусторонние цены на тысячи этих инструментов на крупнейших биржах по всему миру.

- В последние десять минут торговой сессии на бирже Nasdaq маркетмейкеры, такие как Optiver, объединяют данные традиционной книги заказов с данными аукционной книги. Эта возможность консолидировать информацию из обоих источников имеет решающее значение для предоставления наилучших цен всем участникам рынка.

- В этом конкурсе перед вами стоит задача разработать модель, способную предсказать динамику цен на закрытие для сотен акций, котирующихся на бирже Nasdaq, используя данные из книги заказов и аукциона по закрытию акций. Информация с аукциона может быть использована для корректировки цен, оценки динамики спроса и предложения и определения торговых возможностей.

- Ваша модель может способствовать консолидации сигналов с аукциона и книги заявок, что приведет к повышению эффективности и доступности рынка, особенно в течение напряженных последних десяти минут торгов. Вы также получите непосредственный опыт решения реальных задач в области обработки данных


In [ ]:
import os      # Для взаимодействия с операционной системой
import gc      # Для управления сборщиком мусора
import time    # Для работы со временем
import warnings
from warnings import simplefilter  # Для управления предупреждениями
from itertools import combinations  # Для создания комбинаций элементов


import joblib  # Для сериализации и десериализации моделей и других объектов
import numpy as np  # Библиотека для работы с массивами
import pandas as pd  # Библиотека для работы с табличными данными


from catboost import CatBoostRegressor, EShapCalcType, EFeaturesSelectionAlgorithm  # Библиотека CatBoost для построения моделей градиентного бустинга
import lightgbm as lgb  # Библиотека LightGBM для градиентного бустинга
from sklearn.model_selection import KFold, TimeSeriesSplit  # Инструменты для кросс-валидации

from sklearn.metrics import mean_absolute_error  # Метрика для оценки качества моделей

from numba import njit, prange  # Для оптимизации вычислений через JIT-компиляцию

import seaborn as sns  # Библиотека для статистической визуализации
import matplotlib.pyplot as plt  # Библиотека для построения графиков

warnings.filterwarnings("ignore")  # Игнорирование предупреждений
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)  # Игнорирование специфических предупреждений pandas



In [ ]:
!jupyter nbextension enable --py widgetsnbextension

is_offline: Если True, программа работает в офлайновом режиме, что обычно означает использование локальных данных и возможность разделения на обучающие и тестовые наборы данных. Если False, предполагается онлайновый режим, при котором вся доступная информация используется для обучения или инференса без разделения на поднаборы.

is_train: Этот флаг указывает, находится ли программа в режиме обучения. Если True, программа будет выполнять операции, связанные с обучением моделей.

is_infer: Флаг для инференса, то есть применения обученной модели для получения предсказаний на новых данных.

split_day: Это значение используется для разделения временного ряда на различные наборы данных, например, на обучающий и тестовый наборы. В данном случае, все данные до дня 535 будут использоваться для одной цели (например, обучения), а данные после - для другой (например, тестирования или валидации).

In [ ]:
# Настройка параметров
is_offline = False    # Флаг для выбора режима работы: онлайн (False) или офлайн (True)
is_train = True       # Флаг для выбора режима обучения: True для обучения, False для других целей
is_infer = True       # Флаг для выбора режима вывода (инференса): True для вывода, False в противном случае
#***** 435
split_day = 435       # День для разделения временного ряда на обучающий и тестовый/вали

In [ ]:
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
df = df.dropna(subset=["target"])
df.reset_index(drop=True, inplace=True)
df.shape

In [ ]:


# Функция для вычисления дисбаланса трех значений
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        
        for j in range(num_rows):
            # Вычисление максимального, среднего и минимального значений
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            # Предотвращение деления на ноль
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                # Вычисление дисбаланса
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

# Функция для расчета дисбаланса по всем комбинациям троек столбцов
def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Вызов функции compute_triplet_imbalance
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Формирование DataFrame с результатами
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

# Функция для вычисления скользящего среднего
@njit(fastmath=True)
def rolling_average(arr, window):
    n = len(arr)
    result = np.empty(n)
    result[:window] = np.nan  # Заполнение начальных значений NaN
    cumsum = np.cumsum(arr)

    for i in range(window, n):
        result[i] = (cumsum[i] - cumsum[i - window]) / window

    return result

# Функция для параллельного вычисления скользящих средних для разных размеров окон
@njit(parallel=True)
def compute_rolling_averages(df_values, window_sizes):
    num_rows, num_features = df_values.shape
    
    num_windows = len(window_sizes)+14 # -5!!
    rolling_features = np.empty((num_rows, num_features, num_windows))

    for feature_idx in prange(num_features):
        for window_idx, window in enumerate(window_sizes):
            # Вычисление скользящего среднего для каждого окна
            rolling_features[:, feature_idx, window_idx] = rolling_average(df_values[:, feature_idx], window)

    return rolling_features


In [ ]:
if is_offline:
    # Если программа работает в офлайновом режиме, разделить данные на обучающий и валидационный наборы.
    # Разделение происходит на основе значения переменной split_day.
    df_train = df[df["date_id"] <= split_day]
    df_valid = df[df["date_id"] > split_day]
    
    # Вывод сообщения об офлайновом режиме и размерах обучающего и валидационного наборов данных.
    print("Offline mode")
    print(f"train : {df_train.shape}, valid : {df_valid.shape}")
else:
    # Если программа работает в онлайновом режиме, использовать весь набор данных для обучения.
    df_train = df
    
    # Вывод сообщения об онлайновом режиме.
    print("Online mode")


**Этот блок кода используется для создания глобальных характеристик по каждому stock_id в обучающем наборе данных, если программа находится в режиме обучения (is_train).**

In [ ]:
if is_train:
    # Создание словаря глобальных характеристик для каждого stock_id, если находимся в режиме обучения.
    global_stock_id_feats = {
        # Медиана размеров заявок (bid_size + ask_size) для каждого stock_id.
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        # Стандартное отклонение размеров заявок для каждого stock_id.
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        # Размах размеров заявок (максимальный - минимальный) для каждого stock_id.
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        # Медиана цен заявок (bid_price + ask_price) для каждого stock_id.
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        # Стандартное отклонение цен заявок для каждого stock_id.
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        # Размах цен заявок (максимальная - минимальная) для каждого stock_id.
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    }

    

In [ ]:
def imbalance_features_lgbm(df):
    # Определение списков названий столбцов, связанных с ценой и размером
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    # Расчет объема и средней цены
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    # Расчет различных видов дисбаланса
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    # Расчет дисбаланса для всех комбинаций цен
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
    # Расчет дисбаланса для определенных групп столбцов
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    # Расчет дополнительных фичей дисбаланса
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    # Расчет статистических агрегаций
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
    # Создание смещенных и процентных фичей
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    # Расчет разности значений для определенных столбцов
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'market_urgency', 'imbalance_momentum', 'size_imbalance']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
    # Замена бесконечных значений на ноль
    return df.replace([np.inf, -np.inf], 0)

def other_features_lgbm(df):
    # Расчет дней недели, секунд и минут
    df["dow"] = df["date_id"] % 5  # День недели
    df["seconds"] = df["seconds_in_bucket"] % 60
    df["minute"] = df["seconds_in_bucket"] // 60
    # Применение глобальных фичей для каждого stock_id
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())
    return df

def generate_all_features_lgbm(df):
    # Выборка релевантных столбцов для генерации фич
    cols = [c for c in df.columns if c not in ["row_id", "time_id", "target"]]
    df = df[cols]
    # Генерация фич дисбаланса и других фич
    df = imbalance_features_lgbm(df)
    df = other_features_lgbm(df)
    gc.collect()  # Очистка памяти
    # Формирование списка имен фич
    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id"]]
    return df[feature_name]


In [ ]:
def load_models_from_folder(model_save_path, num_folds=5):
    loaded_models = []

    # Загрузка моделей для каждого фолда (части данных)
    for i in range(1, num_folds + 1):
         # Составление пути к файлу модели
        model_filename = os.path.join(model_save_path, f'doblez_{i}.txt')
        # Проверка существования файла модели и его загрузка
        if os.path.exists(model_filename):
            loaded_model = lgb.Booster(model_file=model_filename)
            loaded_models.append(loaded_model)
            print(f"Модель для фолда {i} загружена из {model_filename}")
        else:
            print(f"Файл модели {model_filename} не найден.")

     # Загрузка финальной модели
    final_model_filename = os.path.join(model_save_path, 'doblez-conjunto.txt')
    if os.path.exists(final_model_filename):
        final_model = lgb.Booster(model_file=final_model_filename)
        loaded_models.append(final_model)
        print(f"Финальная модель загружена из {final_model_filename}")
    else:
        print(f"Файл финальной модели {final_model_filename} не найден.")
        
#         # Создание модели CatBoost с параметрами по умолчанию
#     catboost_model = CatBoostRegressor()  # Или CatBoostRegressor для регрессии
#     loaded_models.append(catboost_model)
#     print("Модель CatBoost с параметрами по умолчанию добавлена.")    
    
    return loaded_models

# использования функции для загрузки моделей из списка папок
folders = [
    '/kaggle/input/lightgbm-models/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (1)/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (2)/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (3)/modelitos_para_despues',
     '/kaggle/input/ensemble-of-models/results (4)/modelitos_para_despues',
     '/kaggle/input/ensemble-of-models/results (5)/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (6)/modelitos_para_despues',
    '/kaggle/input/ensemble-of-models/results (7)/modelitos_para_despues',
]
num_folds = 7# 10
all_loaded_models = []
for folder in folders:
    all_loaded_models.extend(load_models_from_folder(folder))

In [ ]:
# моделей
number_of_models = len(all_loaded_models)
print(f"Количество загруженных моделей: {number_of_models}")


In [ ]:
# # названия
# for model in all_loaded_models:
#     print(type(model))


|вес | score | версия |
|-------------|-------------|-------------|
|  0.15   | 5.34   |   7  |
|  0.024  | 5.33  |  8  |
|  0.02   | 5.3388   |   9  |
|  0.021   |5.3389    |  10 |
|  0.012   | 5.3386   |   11  |
|  0.019   |5.3388    |   12  |
|  0.011   |  86   | 13   |
|  0.010   |  86  | 14   |
|  0.009   |  86   | 15   |
|  0.008   |  85   | 16   |
|  0.005   |  85   | 17   |+
|  0.001   |     | 18   |
|  0.002   |     | 19   |
|  0.003   |     | 20   |
|  0.004   |     | 21   |
|  0.005   |     | 22   |



In [ ]:
# Предположим, что all_loaded_models - ваш список моделей
# Начальное определение равных весов для всех моделей
model_weights = [1 / len(all_loaded_models)] * len(all_loaded_models)

# Изменение весов у каждой второй модели
new_weight = 0.005  # Новый вес для каждой второй модели 
for i in range(1, len(all_loaded_models), 2):
    model_weights[i] = new_weight

# Расчет общей суммы весов после изменения
total_weight = sum(model_weights)

# Корректировка весов остальных моделей
if total_weight != 1:
    scale_factor = 1 / total_weight
    model_weights = [w * scale_factor for w in model_weights]

# Проверка, что сумма весов равна 1
assert abs(sum(model_weights) - 1) < 1e-6, "Сумма весов не равна 1"

In [ ]:
# Печать текущих весов
print("Текущие веса моделей:")
for i, weight in enumerate(model_weights):
    print(f"Модель {i}: Вес = {weight}")

In [ ]:
# #  изменить вес первой и третьей моделей
# model_weights[0] = 0.2  # Новый вес для первой модели
# model_weights[2] = 0.15 # Новый вес для третьей модели

In [ ]:
# После изменения весов, корректируем остальные веса, чтобы сумма была равна 1
total_weight = sum(model_weights)
if total_weight != 1:
    scale_factor = 1 / total_weight
    model_weights = [w * scale_factor for w in model_weights]

# Проверка, что сумма весов равна 1
assert abs(sum(model_weights) - 1) < 1e-6, "Сумма весов не равна 1"

In [ ]:
# Печать обновленных весов
print("Обновленные веса моделей:")
for i, weight in enumerate(model_weights):
    print(f"Модель {i}: Вес = {weight}")


In [ ]:
sum(model_weights)

## Submissions

In [ ]:
import time  # Импорт модуля для работы со временем

def zero_sum(prices, volumes):
    # Функция для корректировки предсказаний так, чтобы их сумма была равна нулю
    std_error = np.sqrt(volumes)  # Вычисление стандартной ошибки
    step = np.sum(prices) / np.sum(std_error)  # Расчет шага корректировки
    out = prices - std_error * step  # Корректировка цен
    return out  # Возврат скорректированных цен

# Инференс
if is_infer:
    import optiver2023  # Импорт среды соревнования
    env = optiver2023.make_env()  # Настройка среды для соревнования
    iter_test = env.iter_test()   # Получение итератора для тестового набора данных
    counter = 0                   # Инициализация счетчика
    y_min, y_max = -64, 64        # Установка границ предсказаний
    qps = []                      # Отслеживание запросов в секунду
    cache = pd.DataFrame()        # Инициализация кеша для хранения тестовых данных    
    
    
    
#     # Начальное определение равных весов для всех модел
#      model_weights = [1/len(all_loaded_models)] * len(all_loaded_models)  # Веса моделей
#     # Изменение весов у каждой второй модели
#     new_weight = 0.025  # Новый вес для каждой второй модели
#     for i in range(1, len(all_loaded_models), 2):  # Пропускаем одну модель и изменяем следующую
#         model_weights[i] = new_weight
        
#     # Корректировка весов остальных моделей
#     total_weight = sum(model_weights)
#     for i in range(len(all_loaded_models)):
#         if i % 2 == 0:  # Для моделей, чей вес не был изменен
#             model_weights[i] = model_weights[i] * (1 - total_weight) / sum(model_weights[::2])

#     # Убедимся, что сумма весов равна 1
#     assert abs(sum(model_weights) - 1) < 1e-6, "Сумма весов не равна 1"
    
    
    
    
# #     *******************************************
#   # Начальное задание весов
#     model_weights = [1/len(all_loaded_models)] * len(all_loaded_models)

#     # Изменение весов только для моделей CatBoost


#     # Устанавливаем новый вес для моделей CatBoost и считаем сумму весов
#     new_catboost_weight = 0.05
#     total_weight = 0
#     for i, model in enumerate(all_loaded_models):
#         if isinstance(model, CatBoostRegressor):
#             model_weights[i] = new_catboost_weight
#         total_weight += model_weights[i]

#     # Корректировка весов остальных моделей
#     remaining_weight = 1 - total_weight
#     for i, model in enumerate(all_loaded_models):
#         if not isinstance(model, CatBoostRegressor):
#             model_weights[i] *= remaining_weight / (1 - new_catboost_weight * len([m for m in all_loaded_models if isinstance(m, CatBoostRegressor)]))


#     # Создание DataFrame для документирования изменений
#     columns = ['Model Type', 'Weight', 'Performance Metric']
#     model_info = pd.DataFrame(columns=columns)
    
#     # Заполнение метрик производительности
#     performance_metrics = []
#     catboost_performance = 0.95  # Пример метрики производительности для CatBoost
#     lightgbm_performance = 0.85  # Пример метрики производительности для LightGBM
    
#     for model in all_loaded_models:
#         if isinstance(model, CatBoostRegressor):
#             performance_metrics.append(catboost_performance)
#         else:
#             performance_metrics.append(lightgbm_performance)

#     # Добавление информации о каждой модели
#     rows = []
#     for i, model in enumerate(all_loaded_models):
#         model_type = type(model).__name__
#         weight = model_weights[i]
#         performance = performance_metrics[i]

#         rows.append({'Model Type': model_type,
#                      'Weight': weight,
#                      'Performance Metric': performance})

#     # Обновление DataFrame
#     model_info = pd.concat([model_info, pd.DataFrame(rows)], ignore_index=True)

#     # Вывод таблицы
#     print(model_info)
#     #     *******************************************
     
    
    for (test_df, revealed_targets, sample_prediction_df) in iter_test:
        test_df = test_df.drop('currently_scored', axis=1)  # Удаление лишнего столбца
        now_time = time.time()  # Текущее время для измерения производительности
        print('counter:', counter)
        # Объединение новых тестовых данных с кешем, сохранение последних 21 наблюдений для каждого stock_id
        cache = pd.concat([cache, test_df], ignore_index=True, axis=0)
        if counter > 0:
            cache = cache.groupby('stock_id').tail(21).reset_index(drop=True)
        
        feat = generate_all_features_lgbm(cache)[-len(test_df):]  # Генерация признаков
        pred = model_weights[0] * all_loaded_models[0].predict(feat)  # Предсказание первой модели
        
        # Генерация предсказаний для каждой модели и расчет взвешенного среднего
        for model, weight in zip(all_loaded_models[1:], model_weights[1:]):
            pred += weight * model.predict(feat)
        
        # Применение функции zero_sum и ограничение предсказаний
        pred = zero_sum(pred, test_df['bid_size'] + test_df['ask_size'])
        clipped_predictions = np.clip(pred, y_min, y_max)  # Ограничение предсказаний
        
        # Установка предсказаний в sample_prediction_df
        sample_prediction_df['target'] = clipped_predictions
        
        # Использование среды для выполнения предсказаний
        env.predict(sample_prediction_df)
        
        counter += 1  # Увеличение счетчика
        qps.append(time.time() - now_time)  # Добавление времени запроса
        
        if counter % 10 == 0:
            print(f"{counter} queries per second: {np.mean(qps)}")  # Вывод среднего времени запросов

    time_cost = 1.146 * np.mean(qps)  # Расчет ожидаемого времени выполнения
    print(f"The code will take approximately {np.round(time_cost, 2)} hours to reason about")  # Вывод ожидаемого времени
